In [1]:
import sys
import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf
import platform

print(f"Python Platform: {platform.platform()}")
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Python Platform: macOS-13.2.1-arm64-arm-64bit
Tensor Flow Version: 2.12.0
Keras Version: 2.12.0

Python 3.10.10 | packaged by conda-forge | (main, Mar 24 2023, 20:12:31) [Clang 14.0.6 ]
Pandas 2.1.1
Scikit-Learn 1.3.1
GPU is available


In [2]:
# IMPORTING STATEMENTS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#import mutual_information_regression as mri
#import helper_functions_fs as hf

np.random.seed(7)

In [3]:
from sklearn.preprocessing import StandardScaler

# Import the Data

In [4]:
# training data
data = pd.read_csv('../../0_data/train.csv', index_col=0).reset_index(drop=True)

In [5]:
data.drop(['Borrower2Dummy', 'Bank_infrequent_sklearn', 'LoanPurpose_6','MortgageType_infrequent_sklearn', 
           'Gender1_2', 'Gender2_2', 'PropertyType_infrequent_sklearn', 'Race1_infrequent_sklearn', 'Minority2_White', 
           'Race2_infrequent_sklearn','State_infrequent_sklearn', 'Minority_White'
          ], axis=1, inplace=True)

In [6]:
# defining response variable and input features
y = data.pop('InterestRate')
X = data.copy()

In [7]:
data_test = pd.read_csv('TEST_DATA.csv', index_col=0).reset_index(drop=True)     # modificato
data_test.head()

,InterestRate,MinorityRatio,CensusTractMedFamIncome,MonthlyIncome,HUDMedIncome,LTV,PaymentCount,Age1,Age2,HousingExpense,...,State_31,State_39,State_42,State_48,State_55,State_infrequent_sklearn,Minority_Min,Minority_Missing,Minority_White,Borrower2Dummy
0,4.500,16.936,154125,14425,119600,72.0,360,30,30,35.82,...,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1
1,5.000,22.466,156053,15803,79200,90.0,360,50,50,18.72,...,0.0,0.0,0.0,0.0,0.0,1.0,1,0,0,1
2,2.625,2.467,72944,9724,76600,80.0,180,33,0,17.10,...,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0
3,3.625,14.460,109038,7044,99300,90.0,360,32,33,21.42,...,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1
4,3.750,4.777,143165,16373,109200,72.0,360,49,49,13.86,...,0.0,0.0,0.0,0.0,0.0,1.0,1,0,0,1


In [8]:
data_test.drop(['Borrower2Dummy', 'Bank_infrequent_sklearn', 'LoanPurpose_6','MortgageType_infrequent_sklearn', 
           'Gender1_2', 'Gender2_2', 'PropertyType_infrequent_sklearn', 'Race1_infrequent_sklearn', 'Minority2_White', 
           'Race2_infrequent_sklearn','State_infrequent_sklearn', 'Minority_White'
          ], axis=1, inplace=True)

In [9]:
y_test = data_test.pop('InterestRate')
X_test = data_test.copy()

# Scaling

In [10]:
numerical = pd.read_csv('../../0_data/numerical.csv', index_col=0)
numerical.iloc[:,0]

0                InterestRate
1               MinorityRatio
2     CensusTractMedFamIncome
3               MonthlyIncome
4                HUDMedIncome
5                         LTV
6                PaymentCount
7                        Age1
8                        Age2
9              HousingExpense
10                DebtExpense
11                        PMI
12               CreditScore1
13               CreditScore2
14                   NoteDate
15                 NoteAmount
Name: 0, dtype: object

In [11]:
# separate numerical and dummies

X_num = pd.concat([X.pop(col) for col in list(numerical.iloc[1:,0])], axis=1)
X['BorrowerCount'] = X['BorrowerCount']-1

X_test_num = pd.concat([X_test.pop(col) for col in list(numerical.iloc[1:,0])], axis=1)
X_test['BorrowerCount'] = X_test['BorrowerCount']-1

In [12]:
# scale numerical vars 
ss = StandardScaler()
X_train_num = ss.fit_transform(X_num)
X_test_num = ss.transform(X_test_num)
X_num = pd.DataFrame(X_train_num, columns=list(numerical.iloc[1:,0]))
X_test_num = pd.DataFrame(X_test_num, columns=list(numerical.iloc[1:,0]))
X_num.describe()

,MinorityRatio,CensusTractMedFamIncome,MonthlyIncome,HUDMedIncome,LTV,PaymentCount,Age1,Age2,HousingExpense,DebtExpense,PMI,CreditScore1,CreditScore2,NoteDate,NoteAmount
count,1.890790e+05,1.890790e+05,1.890790e+05,1.890790e+05,1.890790e+05,1.890790e+05,1.890790e+05,1.890790e+05,1.890790e+05,1.890790e+05,1.890790e+05,1.890790e+05,1.890790e+05,1.890790e+05,1.890790e+05
mean,-2.335544e-16,4.329118e-17,6.898592e-17,-1.819958e-16,6.964825e-16,2.689164e-16,-2.288664e-16,4.235170e-17,1.323913e-16,6.650381e-16,3.112493e-17,2.442832e-16,2.375942e-16,2.158467e-14,-4.906427e-17
std,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00
min,-9.829129e-01,-2.968603e+00,-1.541693e+00,-3.136269e+00,-8.617748e-01,-3.667909e+00,-2.003581e+00,-1.101219e+00,-2.410365e+00,-3.148844e+00,-4.158231e-01,-4.275040e+00,-2.317892e+00,-7.010553e+00,-1.761821e+00
25%,-6.899229e-01,-6.907823e-01,-6.998868e-01,-6.839963e-01,-8.617748e-01,-2.778056e-01,-8.066413e-01,-1.101219e+00,-7.202951e-01,-7.560665e-01,-4.158231e-01,-5.143845e-01,-5.737875e-01,-1.000778e+00,-7.500204e-01
50%,-3.806037e-01,-2.074440e-01,-2.267679e-01,-6.296614e-02,-9.168695e-02,5.697203e-01,-1.333627e-01,1.920703e-01,-1.531912e-01,8.581993e-03,-4.158231e-01,7.391671e-01,-5.737875e-01,2.011766e-01,-2.017936e-01
75%,2.836096e-01,5.193869e-01,4.021168e-01,5.102925e-01,1.650090e-01,5.697203e-01,6.895334e-01,7.761363e-01,5.862923e-01,7.836339e-01,-4.158231e-01,7.391671e-01,1.170317e+00,1.403131e+00,5.519377e-01
max,3.195775e+00,4.143399e+00,3.701777e+00,3.949844e+00,5.170580e+00,5.697203e-01,3.981118e+00,2.862086e+00,4.372398e+00,5.361121e+00,7.908332e+00,7.391671e-01,1.170317e+00,1.403131e+00,4.433948e+00


In [13]:
# merge back together numerical and dummies
X = pd.concat([X, X_num], axis=1, ignore_index=False)
X_test = pd.concat([X_test, X_test_num], axis=1,ignore_index=False)

In [14]:
X_test

,BorrowerCount,FirstTime,SelfEmployed,Bank_Boston,Bank_Chicago,Bank_Cincinnati,Bank_Dallas,Bank_Des Moines,Bank_Indianapolis,Bank_Pittsburgh,...,PaymentCount,Age1,Age2,HousingExpense,DebtExpense,PMI,CreditScore1,CreditScore2,NoteDate,NoteAmount
0,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.569720,-1.105876,0.150351,2.064010,1.740745,-0.415823,-0.514385,-0.573787,-1.000778,3.998591
1,1,0,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.569720,0.390298,0.984731,-0.071998,0.784674,2.252175,-3.021488,-1.009814,-1.000778,1.171192
2,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,-1.972857,-0.881450,-1.101219,-0.274357,0.940725,-0.415823,-0.514385,1.170317,1.403131,-0.035713
3,1,1,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.569720,-0.956259,0.275508,0.265267,-0.623944,2.252175,-0.514385,-0.573787,-1.000778,-0.040550
4,1,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.569720,0.315490,0.943012,-0.679074,-0.677001,-0.415823,-0.514385,-1.009814,-1.000778,0.854349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4495,1,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.569720,-0.806641,0.317227,-0.068251,0.238497,-0.415823,0.739167,-0.573787,-1.000778,1.854057
4496,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.569720,-1.554729,-1.101219,0.512594,-0.035112,-0.415823,-0.514385,1.170317,1.403131,0.049746
4497,1,0,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,-1.972857,-0.657024,0.358946,0.565057,0.335248,-0.415823,0.739167,-0.573787,-1.000778,-0.282415
4498,1,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-1.972857,0.240681,1.026450,-1.168732,-0.061121,-0.415823,0.739167,-0.573787,1.403131,-1.048320


In [15]:
X

,BorrowerCount,FirstTime,SelfEmployed,Bank_Boston,Bank_Chicago,Bank_Cincinnati,Bank_Dallas,Bank_Des Moines,Bank_Indianapolis,Bank_Pittsburgh,...,PaymentCount,Age1,Age2,HousingExpense,DebtExpense,PMI,CreditScore1,CreditScore2,NoteDate,NoteAmount
0,1,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.569720,1.512429,1.443640,0.307737,1.138389,-0.415823,-3.021488,-1.881866,-1.000778,-0.677461
1,0,1,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.569720,-0.806641,-1.101219,1.429453,1.267391,-0.415823,0.739167,1.170317,-1.000778,-0.814518
2,1,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.569720,-0.956259,1.235045,1.070954,0.463210,-0.415823,-1.767936,-1.009814,0.201177,0.654875
3,1,1,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.569720,-1.479920,-0.058244,-1.286150,-2.108506,-0.415823,0.739167,-0.573787,0.201177,-0.806455
4,0,0,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.569720,-1.105876,-1.101219,-0.911412,-0.756066,-0.415823,-0.514385,1.170317,-1.000778,-1.274061
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189074,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.569720,0.614725,0.859574,-0.985110,1.531637,2.785775,-0.514385,-1.009814,-1.000778,0.366589
189075,1,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.569720,-0.507406,0.484103,-0.893924,-1.587297,-0.415823,0.739167,-0.573787,1.403131,1.138540
189076,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.569720,-1.554729,-0.099963,-0.986359,-0.797680,-0.415823,-0.514385,-1.009814,1.403131,-0.931419
189077,1,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.569720,-0.657024,0.567541,3.133263,1.758431,-0.415823,-1.767936,-1.445840,-1.000778,-0.201794


In [16]:
# dependent variables
ss_y = StandardScaler()
y = ss_y.fit_transform(np.array(y).reshape(-1, 1))
y_test = ss_y.transform(np.array(y_test).reshape(-1, 1))
pd.Series(y.flatten()).describe()

count    1.890790e+05
mean    -1.981173e-16
std      1.000003e+00
min     -2.684678e+00
25%     -7.623710e-01
50%     -1.856790e-01
75%      5.832437e-01
max      4.427857e+00
dtype: float64

In [17]:
y_test

array([[ 1.73662773],
       [ 2.50555042],
       [-1.14683236],
       ...,
       [-0.57014035],
       [-0.95460169],
       [-0.76237102]])

# Scoring Methods


In [18]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error


def mean_residual(y_true, prediction):
    '''Compute the mean residual metric'''
    residuals = (y_true - prediction)
    MR = residuals.mean()
    return MR


def compute_scores(x, y_true, y_pred):
    '''Compute the following metrics for the given model: MSE, MAE, MR, r2'''
    y_true = y_true.reshape(y_pred.shape)

    MSE = mean_squared_error(y_true, y_pred)
    print(f"MSE: {MSE:.4f}")
    MAE = mean_absolute_error(y_true, y_pred)
    print(f"MAE: {MAE:.4f}")
    MR = mean_residual(y_true, y_pred)
    print(f"MR: {MR:.4f}")
    R2 = r2_score(y_true, y_pred)
    adj_R2 = 1- ((1-R2) * (x.shape[0]-1)/(x.shape[0]-x.shape[1]-1))
    print(f"adj-R2: {adj_R2:.4f}")
    return [MSE, MAE, MR, adj_R2]


def save_scores(model, scores):
    '''Store scores for the given model into a dedicated dataframe for a final comparison across models'''
    MSE, MAE, MR, r2 = scores
    #pd.Series(scores).to_csv('scores/'+model)
    model_names.append(model)
    MSEs.append(MSE)
    MAEs.append(MAE)
    MRs.append(MR)
    r2s.append(r2)
    

def display_scores():
    scores_df = pd.DataFrame([model_names, MSEs, MAEs, MRs, r2s], 
                 index=['method', 'MSE', 'MAE', 'MR', 'adj R-squared']).T
    return scores_df

In [19]:
model_names, MSEs, MAEs, MRs, r2s = [],[],[],[],[]

In [20]:
def export_pred(y_pred_std, model, scaler=ss_y):
    '''Reverts model predictions to the original scale of the input data and then saves it to a csv file'''

    y_pred = scaler.inverse_transform(np.array(y_pred_std, ndmin=2)).flatten()
    np.savetxt('predictions/'+str(model)+'_pred.csv', y_pred, delimiter=",")

# MODELS

# 1. Linear Regression

In [21]:
import statsmodels.api as sm

In [22]:
#add constant to predictor variables
x = sm.add_constant(X)
#fit linear regression model
lr = sm.OLS(y, x).fit()

#view model summary
print(lr.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.705
Model:                            OLS   Adj. R-squared:                  0.705
Method:                 Least Squares   F-statistic:                     7530.
Date:                Tue, 17 Oct 2023   Prob (F-statistic):               0.00
Time:                        10:53:04   Log-Likelihood:            -1.5286e+05
No. Observations:              189079   AIC:                         3.059e+05
Df Residuals:                  189018   BIC:                         3.065e+05
Df Model:                          60                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

In [23]:
X_TEST = sm.add_constant(X_test)
lr_pred = lr.predict(X_TEST)
scores = compute_scores(X_TEST, y_test, lr_pred)
save_scores('LinearRegression', scores)

MSE: 37810656626585387008.0000
MAE: 4772386889.3324
MR: 3978776597.7021
adj-R2: -38603428835928367104.0000


In [24]:
export_pred(lr_pred, 'LinearRegression')

In [25]:
lr_pred

0      -4.956168e+09
1      -4.956168e+09
2      -4.956168e+09
3      -4.956168e+09
4      -4.956168e+09
            ...     
4495    6.314268e-01
4496   -7.414331e-01
4497    1.656870e-01
4498   -1.670942e+00
4499   -7.370813e-01
Length: 4500, dtype: float64

# 5. Random Forest - only predict

In [26]:
yf= y.flatten()

In [27]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [28]:
best_forest = RandomForestRegressor(max_depth=30, n_estimators=128)
best_forest.fit(X, yf)

RandomForestRegressor(max_depth=30, n_estimators=128)

In [29]:
forest_pred = best_forest.predict(X_test)
scores = compute_scores(X_test, y_test, forest_pred)
save_scores('RandomForest', scores)

MSE: 0.1806
MAE: 0.3260
MR: -0.0106
adj-R2: 0.8157


In [30]:
len(forest_pred)

4500

In [31]:
export_pred(forest_pred, 'RandomForest')

# 6. XGBoost 

First we explore the default parameters, then move on to cross validation.

In [32]:
from xgboost import XGBRegressor
import time

In [33]:
best_params_xgb = {'colsample_bytree': 1.0, 'gamma': 0.2, 
                'learning_rate': 0.3, 'max_depth': 6, 
                'min_child_weight': 2, 'n_estimators': 128, 
                'reg_alpha': 0.5, 'reg_lambda': 1, 'subsample': 1.0}

In [34]:
# Creating a XGB model with the selected parameters
best_xgbr = XGBRegressor(max_depth=best_params_xgb['max_depth'],
                        n_estimators=best_params_xgb['n_estimators'],
                        learning_rate=best_params_xgb['learning_rate'],
                        min_child_weight=best_params_xgb['min_child_weight'], 
                        gamma=best_params_xgb['gamma'],
                        alpha=best_params_xgb['reg_alpha'],
                        reg_lambda=best_params_xgb['reg_lambda'],
                        subsample=best_params_xgb['subsample'],
                        colsample_bytree=best_params_xgb['colsample_bytree']
                        )
best_xgbr.fit(X, yf)

/Users/fra/miniconda3/envs/tensorflow/lib/python3.10/site-packages/xgboost/data.py:312: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/Users/fra/miniconda3/envs/tensorflow/lib/python3.10/site-packages/xgboost/data.py:314: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(dtype) and enable_categorical:
/Users/fra/miniconda3/envs/tensorflow/lib/python3.10/site-packages/xgboost/data.py:345: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype)
/Users/fra/miniconda3/envs/tensorflow/lib/python3.10/site-packages/xgboost/data.py:336: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, Categorical

XGBRegressor(alpha=0.5, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=1.0, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.2, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.3, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=2, missing=nan, monotone_constraints=None,
             n_estimators=128, n_jobs=None, num_parallel_tree=None,
             predictor=None, ...)

In [35]:
xgb_pred = best_xgbr.predict(X_test)
scores = compute_scores(X_test, y_test, xgb_pred)

save_scores('XGBoost', scores)

MSE: 0.1722
MAE: 0.3191
MR: -0.0046
adj-R2: 0.8243


/Users/fra/miniconda3/envs/tensorflow/lib/python3.10/site-packages/xgboost/data.py:312: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/Users/fra/miniconda3/envs/tensorflow/lib/python3.10/site-packages/xgboost/data.py:314: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(dtype) and enable_categorical:
/Users/fra/miniconda3/envs/tensorflow/lib/python3.10/site-packages/xgboost/data.py:345: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype)
/Users/fra/miniconda3/envs/tensorflow/lib/python3.10/site-packages/xgboost/data.py:336: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, Categorical

In [36]:
export_pred(xgb_pred, 'XGBoost')

# 7. Light GBM

In [37]:
from lightgbm import LGBMRegressor

Best LGBMRegressor with params: {'colsample_bytree': 1.0, 'learning_rate': 0.3, 'max_depth': 6, 'min_child_weight': 1, 'n_estimators': 128, 'num_leaves': 64, 'reg_alpha': 0.5, 'reg_lambda': 1} and MSE: -0.176


In [38]:
best_params_lgbm= {'colsample_bytree': 1.0, 'learning_rate': 0.3, 
              'max_depth': 6, 'min_child_weight': 1, 'n_estimators': 128, 
               'reg_alpha': 0.5, 'reg_lambda': 1, 'num_leaves': 64} 

In [39]:
# Creating a XGB model with the selected parameters
best_lgbm = LGBMRegressor(max_depth=best_params_lgbm['max_depth'],
                        n_estimators=best_params_lgbm['n_estimators'],
                        learning_rate=best_params_lgbm['learning_rate'],
                        min_child_weight=best_params_lgbm['min_child_weight'], 
                        reg_alpha=best_params_lgbm['reg_alpha'],
                        reg_lambda=best_params_lgbm['reg_lambda'],
                        colsample_bytree=best_params_lgbm['colsample_bytree'],
                        num_leaves=best_params_lgbm['num_leaves'],
                        )
best_lgbm.fit(X, yf)

LGBMRegressor(learning_rate=0.3, max_depth=6, min_child_weight=1,
              n_estimators=128, num_leaves=64, reg_alpha=0.5, reg_lambda=1)

In [40]:
lgbm_pred = best_lgbm.predict(X_test)
scores = compute_scores(X_test, y_test, lgbm_pred)

save_scores('LightGBM', scores)

MSE: 0.1720
MAE: 0.3196
MR: -0.0017
adj-R2: 0.8244


In [41]:
export_pred(lgbm_pred, 'LightGBM')

# 8. Hybrid Regression

In [42]:
# Import predictions from csv files, which have been rescaled to the original scale
forest_pred = pd.read_csv('predictions/RandomForest_pred.csv', header=None)
xgbr_pred = pd.read_csv('predictions/XGBoost_pred.csv', header=None)
lgbm_pred = pd.read_csv('predictions/LightGBM_pred.csv', header=None)

In [43]:
# Define the weights for each model
weights = [0.2, 0.4, 0.4]  # Equal weights for each model

# Combine predictions using weighted average
hybrid_predictions = (weights[0] * forest_pred + 
                      weights[1] * xgbr_pred +     
                      weights[2] * lgbm_pred)

# Standardize, scale to the same scale of y_test
hybrid_predictions = ss_y.transform(np.array(hybrid_predictions).reshape(-1, 1))

In [44]:
scores = compute_scores(X_test, y_test, hybrid_predictions)
save_scores('HybridModel', scores)

MSE: 0.1674
MAE: 0.3145
MR: -0.0046
adj-R2: 0.8291


In [45]:
export_pred(hybrid_predictions, 'HybridModel')

# 9. Stacking
Architecture from Quang

In [46]:
from vecstack import StackingTransformer

# Initialize 1st level estimators
estimators = [('RF', RandomForestRegressor(max_depth=30, #forest.best_params_['max_depth'], 
                                    n_estimators= 128 )),#forest.best_params_['n_estimators'])),
              ('LGBM', LGBMRegressor(max_depth=best_params_lgbm['max_depth'],
                        n_estimators=best_params_lgbm['n_estimators'],
                        learning_rate=best_params_lgbm['learning_rate'],
                        min_child_weight=best_params_lgbm['min_child_weight'], 
                        reg_alpha=best_params_lgbm['reg_alpha'],
                        reg_lambda=best_params_lgbm['reg_lambda'],
                        colsample_bytree=best_params_lgbm['colsample_bytree'],
                        num_leaves=best_params_lgbm['num_leaves'],
              ))]
              
# Initialize StackingTransformer
stack = StackingTransformer(estimators, regression=True, verbose=2, shuffle=True, metric=mean_squared_error)

In [47]:
stack = stack.fit(X, yf)

task:         [regression]
metric:       [mean_squared_error]
variant:      [A]
n_estimators: [2]

estimator  0: [RF: RandomForestRegressor]
    fold  0:  [0.18637706]
    fold  1:  [0.18592003]
    fold  2:  [0.18886899]
    fold  3:  [0.18794788]
    ----
    MEAN:     [0.18727849] + [0.00118699]

estimator  1: [LGBM: LGBMRegressor]
    fold  0:  [0.17716738]
    fold  1:  [0.17598168]
    fold  2:  [0.17829741]
    fold  3:  [0.17618286]
    ----
    MEAN:     [0.17690733] + [0.00091946]



In [48]:
# Get stacked features
S_train = stack.transform(X)
S_test = stack.transform(X_test)

Train set was detected.
Transforming...

estimator  0: [RF: RandomForestRegressor]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE

estimator  1: [LGBM: LGBMRegressor]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE

Transforming...

estimator  0: [RF: RandomForestRegressor]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE

estimator  1: [LGBM: LGBMRegressor]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE



In [49]:
xgbr = XGBRegressor()
start = time.time()
xgbr.fit(S_train, yf)
elapsed = time.time() - start
print(f'XGB Stacking Training ran in {elapsed:.5f} seconds')

XGB Stacking Training ran in 2.66697 seconds


In [50]:
stacking_pred = xgbr.predict(S_test)
scores = compute_scores(S_test, y_test, stacking_pred)

save_scores('Stacking', scores)

MSE: 0.1688
MAE: 0.3165
MR: -0.0084
adj-R2: 0.8299


In [51]:
export_pred(stacking_pred, 'Stacking')

# FINAL SUMMARY OF METHODS PERFORMANCES

We display below a summary of the scores obtained by the different models. 
We see that models can be classified differently based on the metric of evaluation chosen. 




In [52]:
final_scores=display_scores()

In [53]:
final_scores

,method,MSE,MAE,MR,adj R-squared
0,LinearRegression,37810656626585387008.0,4772386889.332389,3978776597.702141,-38603428835928367104.0
1,RandomForest,0.180579,0.326028,-0.010638,0.815677
2,XGBoost,0.172155,0.319065,-0.004571,0.824275
3,LightGBM,0.172,0.319619,-0.001681,0.824434
4,HybridModel,0.167448,0.314496,-0.004628,0.82908
5,Stacking,0.168848,0.316522,-0.008392,0.829912


In [54]:
final_scores.to_csv('final_scores.csv')

# analisi finali
- predictions per anno vs reale (grafica)